In [4]:
import pandas as pd
import sys
import json
import requests

In [6]:
df = pd.read_csv("/home/jovyan/work/Treatment-normalized_drugs.csv")

In [7]:
df.head()

,bcr_patient_uuid,bcr_patient_barcode,bcr_drug_barcode,bcr_drug_uuid,form_completion_date,pharmaceutical_therapy_drug_name,clinical_trial_drug_classification,pharmaceutical_therapy_type,pharmaceutical_tx_started_days_to,pharmaceutical_tx_ongoing_indicator,...,prescribed_dose,regimen_number,route_of_administration,stem_cell_transplantation,stem_cell_transplantation_type,therapy_regimen,therapy_regimen_other,total_dose,tx_on_clinical_trial,normalized_drug
0,92554413-9EBC-4354-8E1B-9682F3A031D9,TCGA-4N-A93T,TCGA-4N-A93T-D65957,25E5CD87-BB8D-4D2A-8219-1660D2F13AEE,2014-10-01 00:00:00,Xeloda,[Not Available],Chemotherapy,68,YES,...,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Applicable],[Not Available],NO,capecitabine
1,565e2726-4942-4726-89d3-c5e3797f7204,TCGA-A6-2671,TCGA-A6-2671-D6020,3aefce87-3734-4b5f-a4e2-c62dbba5badf,2011-01-11 00:00:00,Study drug AMG 655,[Not Available],"Other, specify in notes",96,NO,...,[Not Available],1,IV,[Not Available],[Not Available],PALLIATIVE,[Not Applicable],[Not Available],[Not Available],conatumumab
2,565e2726-4942-4726-89d3-c5e3797f7204,TCGA-A6-2671,TCGA-A6-2671-D6055,409b319f-5642-4f38-8e5d-d948107d0425,2011-01-12 00:00:00,5 FU,[Not Available],Chemotherapy,96,NO,...,450-735,1,IV,[Not Available],[Not Available],PALLIATIVE,[Not Applicable],7185,[Not Available],fluorouracil
3,565e2726-4942-4726-89d3-c5e3797f7204,TCGA-A6-2671,TCGA-A6-2671-D6059,dc9ad6a9-f324-4d56-93ae-dad6cd659a43,2011-01-12 00:00:00,Irinotecan,[Not Available],Chemotherapy,551,YES,...,320-325,1,IV,[Not Available],[Not Available],PROGRESSION,[Not Applicable],2250,[Not Available],irinotecan
4,565e2726-4942-4726-89d3-c5e3797f7204,TCGA-A6-2671,TCGA-A6-2671-D6049,891f0411-bdcc-468d-bc50-acc539d6bc11,2011-01-12 00:00:00,Leucovorin,[Not Available],"Other, specify in notes",96,NO,...,700-735,1,IV,[Not Available],[Not Available],PALLIATIVE,[Not Applicable],8435,[Not Available],leucovorin


In [10]:
len(df)

593

In [39]:
def agg_drug(drug_series):
    return "|".join(drug_series.sort_values().unique())

In [13]:
df[df["bcr_patient_uuid"].duplicated()]

,bcr_patient_uuid,bcr_patient_barcode,bcr_drug_barcode,bcr_drug_uuid,form_completion_date,pharmaceutical_therapy_drug_name,clinical_trial_drug_classification,pharmaceutical_therapy_type,pharmaceutical_tx_started_days_to,pharmaceutical_tx_ongoing_indicator,...,prescribed_dose,regimen_number,route_of_administration,stem_cell_transplantation,stem_cell_transplantation_type,therapy_regimen,therapy_regimen_other,total_dose,tx_on_clinical_trial,normalized_drug
2,565e2726-4942-4726-89d3-c5e3797f7204,TCGA-A6-2671,TCGA-A6-2671-D6055,409b319f-5642-4f38-8e5d-d948107d0425,2011-01-12 00:00:00,5 FU,[Not Available],Chemotherapy,96,NO,...,450-735,1,IV,[Not Available],[Not Available],PALLIATIVE,[Not Applicable],7185,[Not Available],fluorouracil
3,565e2726-4942-4726-89d3-c5e3797f7204,TCGA-A6-2671,TCGA-A6-2671-D6059,dc9ad6a9-f324-4d56-93ae-dad6cd659a43,2011-01-12 00:00:00,Irinotecan,[Not Available],Chemotherapy,551,YES,...,320-325,1,IV,[Not Available],[Not Available],PROGRESSION,[Not Applicable],2250,[Not Available],irinotecan
4,565e2726-4942-4726-89d3-c5e3797f7204,TCGA-A6-2671,TCGA-A6-2671-D6049,891f0411-bdcc-468d-bc50-acc539d6bc11,2011-01-12 00:00:00,Leucovorin,[Not Available],"Other, specify in notes",96,NO,...,700-735,1,IV,[Not Available],[Not Available],PALLIATIVE,[Not Applicable],8435,[Not Available],leucovorin
5,565e2726-4942-4726-89d3-c5e3797f7204,TCGA-A6-2671,TCGA-A6-2671-D6091,75929a1b-da0b-4859-9592-e0b096b3e46c,2011-01-12 00:00:00,5 FU,[Not Available],Chemotherapy,551,YES,...,725,2,IV,[Not Available],[Not Available],PROGRESSION,[Not Applicable],5075,[Not Available],fluorouracil
6,565e2726-4942-4726-89d3-c5e3797f7204,TCGA-A6-2671,TCGA-A6-2671-D6025,b7b6c0f7-bf3f-45c3-b847-79b03842ba16,2011-01-12 00:00:00,Avastin,[Not Available],Targeted Molecular therapy,96,NO,...,300-320,1,IV,[Not Available],[Not Available],PALLIATIVE,[Not Applicable],3620,[Not Available],bevacizumab
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,5F6ED48B-3B5A-4D20-8FBF-C9AA3FEEA186,TCGA-RU-A8FL,TCGA-RU-A8FL-D68258,35B51064-B60A-468F-A98F-BAC6F14F516D,2014-11-14 00:00:00,Irinotecan + Cetuximab,[Not Available],Chemotherapy,343,NO,...,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Applicable],[Not Available],NO,cetuximab|irinotecan
588,5F6ED48B-3B5A-4D20-8FBF-C9AA3FEEA186,TCGA-RU-A8FL,TCGA-RU-A8FL-D68257,09D26C80-A0A1-453B-901C-347775472D0F,2014-11-14 00:00:00,FOLFIRI/Avastin,[Not Available],Chemotherapy,270,NO,...,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Applicable],[Not Available],NO,bevacizumab|fluorouracil|leucovorin|irinotecan
589,5F6ED48B-3B5A-4D20-8FBF-C9AA3FEEA186,TCGA-RU-A8FL,TCGA-RU-A8FL-D68088,6F9A2902-54C7-401D-BEEB-5A1F444AFCCE,2014-11-14 00:00:00,FOLFOX,[Not Available],Chemotherapy,48,NO,...,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Applicable],[Not Available],NO,fluorouracil|leucovorin|oxaliplatin
591,8D0FA506-3F51-4108-8427-7DABF270BCD4,TCGA-SS-A7HO,TCGA-SS-A7HO-D71979,6E855AC1-15F0-41DD-995C-21AA794684C2,2015-03-31 00:00:00,5-FU,[Not Available],Chemotherapy,37,NO,...,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Applicable],[Not Available],NO,fluorouracil


In [34]:
df['pharmaceutical_tx_started_days_to'] = pd.to_numeric(df['pharmaceutical_tx_started_days_to'],
                  errors='coerce')

In [40]:
agg = df[df["bcr_patient_uuid"].duplicated()] \
  .groupby(['bcr_patient_uuid', 'bcr_patient_barcode', 'pharmaceutical_tx_started_days_to'], as_index=False) \
  .agg({'normalized_drug': agg_drug}) \
  .sort_values(by=['bcr_patient_barcode', 'pharmaceutical_tx_started_days_to'], ascending=[True, True])
               
agg

,bcr_patient_uuid,bcr_patient_barcode,pharmaceutical_tx_started_days_to,normalized_drug
56,565e2726-4942-4726-89d3-c5e3797f7204,TCGA-A6-2671,96.0,bevacizumab|fluorouracil|leucovorin|oxaliplatin
57,565e2726-4942-4726-89d3-c5e3797f7204,TCGA-A6-2671,551.0,bevacizumab|fluorouracil|irinotecan
58,565e2726-4942-4726-89d3-c5e3797f7204,TCGA-A6-2671,725.0,leucovorin
59,565e2726-4942-4726-89d3-c5e3797f7204,TCGA-A6-2671,844.0,bevacizumab|fluorouracil|irinotecan|leucovorin
60,565e2726-4942-4726-89d3-c5e3797f7204,TCGA-A6-2671,1041.0,cetuximab|irinotecan
...,...,...,...,...
120,9F45A227-1FD5-478C-A194-4A5C61B68CC3,TCGA-QG-A5YW,42.0,oxaliplatin
67,5F6ED48B-3B5A-4D20-8FBF-C9AA3FEEA186,TCGA-RU-A8FL,48.0,fluorouracil|leucovorin|oxaliplatin
68,5F6ED48B-3B5A-4D20-8FBF-C9AA3FEEA186,TCGA-RU-A8FL,270.0,bevacizumab|fluorouracil|leucovorin|irinotecan
69,5F6ED48B-3B5A-4D20-8FBF-C9AA3FEEA186,TCGA-RU-A8FL,343.0,cetuximab|irinotecan


In [41]:
agg.to_csv('./TGCAclinical_treatment_grouped.csv')

ValueError: invalid literal for int() with base 10: '[Not Available]'

In [21]:
agg[agg['bcr_patient_uuid'] == 'fce1fd5a-54d2-4260-b187-4eb7035e96e9']

,bcr_patient_uuid,bcr_patient_barcode,pharmaceutical_tx_started_days_to,normalized_drug
195,fce1fd5a-54d2-4260-b187-4eb7035e96e9,TCGA-CM-6675,24,fluorouracil|leucovorin|oxaliplatin
196,fce1fd5a-54d2-4260-b187-4eb7035e96e9,TCGA-CM-6675,337,floxuridine|fluorouracil|irinotecan|leucovorin
197,fce1fd5a-54d2-4260-b187-4eb7035e96e9,TCGA-CM-6675,366,cetuximab
198,fce1fd5a-54d2-4260-b187-4eb7035e96e9,TCGA-CM-6675,54,pegfilgrastim
